# Pretty Little Thing

### Importing libraries

In [3]:
from bs4 import BeautifulSoup
import lxml
import requests
import csv
import datetime
import re
from math import *

### Creation of the url list

In [ ]:
url_final=[]

In [ ]:
url="https://www.prettylittlething.us/clothing.html"
response = requests.get(url)
if response.status_code == 200:
    print("Success")
else:
    print("Failure")
ipp=[]
i=[]
results_page = BeautifulSoup(response.content,'lxml')
items_per_page = results_page.find('span', class_="numberOfListerItemsDisplayed")
ipp+=items_per_page
ipp=int(ipp[0])
items=results_page.find('span', class_="numberOfListerItems")
i+=items
i=int(i[0])
number_of_pages=ceil(i/ipp)
p=["https://www.prettylittlething.us/clothing.html?id=65&page="+str(i)+"&ajax="+str(int(i/10)) for i in range(1, number_of_pages+1)]
url_final+=p
print(number_of_pages)
print( url_final)

Success
516
['https://www.prettylittlething.us/clothing.html?id=65&page=150&ajax=15', 'https://www.prettylittlething.us/clothing.html?id=65&page=151&ajax=15', 'https://www.prettylittlething.us/clothing.html?id=65&page=152&ajax=15', 'https://www.prettylittlething.us/clothing.html?id=65&page=153&ajax=15', 'https://www.prettylittlething.us/clothing.html?id=65&page=154&ajax=15', 'https://www.prettylittlething.us/clothing.html?id=65&page=155&ajax=15', 'https://www.prettylittlething.us/clothing.html?id=65&page=156&ajax=15', 'https://www.prettylittlething.us/clothing.html?id=65&page=157&ajax=15', 'https://www.prettylittlething.us/clothing.html?id=65&page=158&ajax=15', 'https://www.prettylittlething.us/clothing.html?id=65&page=159&ajax=15', 'https://www.prettylittlething.us/clothing.html?id=65&page=160&ajax=16', 'https://www.prettylittlething.us/clothing.html?id=65&page=161&ajax=16', 'https://www.prettylittlething.us/clothing.html?id=65&page=162&ajax=16', 'https://www.prettylittlething.us/clot

In [ ]:
url="https://www.prettylittlething.us/shoes.html"
response = requests.get(url)
if response.status_code == 200:
    print("Success")
else:
    print("Failure")
ipp=[]
i=[]
results_page = BeautifulSoup(response.content,'lxml')
items_per_page = results_page.find('span', class_="numberOfListerItemsDisplayed")
ipp+=items_per_page
ipp=int(ipp[0])
items=results_page.find('span', class_="numberOfListerItems")
i+=items
i=int(i[0])
number_of_pages=ceil(i/ipp)
p=["https://www.prettylittlething.us/shoes.html?id=32&page="+str(i)+"&ajax="+str(int(i/10)) for i in range(1,number_of_pages+1)]
url_final+=p

Success


In [ ]:
url="https://www.prettylittlething.us/accessories.html"
response = requests.get(url)
if response.status_code == 200:
    print("Success")
else:
    print("Failure")
ipp=[]
i=[]
results_page = BeautifulSoup(response.content,'lxml')
items_per_page = results_page.find('span', class_="numberOfListerItemsDisplayed")
ipp+=items_per_page
ipp=int(ipp[0])
items=results_page.find('span', class_="numberOfListerItems")
i+=items
i=int(i[0])
number_of_pages=ceil(i/ipp)
p=["https://www.prettylittlething.us/accessories.html?id=18&page="+str(i)+"&ajax=" + str(int(i/10)) for i in range(1,number_of_pages+1)]
url_final+=p

Success


### Creation of a CSV file with web scraped data

In [ ]:
# CSV file creation

csv_file=open('PrettyLittleThing_table.csv','w')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['id_style', 'display_name', 'materials', 'color', 'size', 'price', 'currency', 'product_url', 'image_links', 'brand_name', 'retailer', 'description', 'scrapped_date', 'high_level', 'low_level', 'gender', 'secondhand', 'shipping_from', 'style'])

#First loop to browse all urls

for url in url_final:
    response = requests.get(url)
    if response.status_code == 200:
        print("Success")
    else:
        print("Failure")
    results_page = BeautifulSoup(response.content,'lxml')
    
#Second loop to browse all the products on the page    
    
    for tag in results_page.find_all('div', class_="category-product js-productitem js-select-productimpression"):
        display_name=tag['data-productname']
        id_style=float("Nan")
        shipping_from=float("Nan")
        style=float("Nan")
        currency='USD'
        color=tag['data-colour']
        price=tag['data-price']
        brand_name='PrettyLittleThing'
        retailer='PrettyLittleThing'
        description=[]
        materials=float("NaN")
        scrapped_date=str(datetime.date.today())
        high_level=float("Nan")
        low_level=tag['data-style']
        gender='Female'
        secondhand='No'
        url=tag.find('a', class_="product-url js-click-product js-category-product__link")
        product_url=url['href']
        image=tag.find('img', class_="lazyload")
        image_links=image['data-src']
        info = requests.get(product_url)
        result_info = BeautifulSoup(info.content,'lxml')
        try:
            description=result_info.find('div', class_="target-description").p.text
            materials=result_info.find('div', class_="accordion-text").p.text
        except Exception as e:
            pass
        l=[]
        
#Third loop to extract the information on the page of each product        
        try:
          price=result_info.find('p', class_="price-container new").span.text
        except Exception as e:
          pass
        for ta in result_info.find_all('div',class_="plt-box size-option product-size-select singleWidth size-in-stock saleable js-click-sizeoption js-selectsize-option"):
            l+=ta
            for i in range(len(l)):
                k=re.findall('[a-zA-Z_0-9]+', str(l[i]))
                l[i]=k
        size = [item for sublist in l for item in sublist]
        csv_writer.writerow([id_style, display_name, materials, color, size, price, currency, product_url, image_links, brand_name, retailer, description, scrapped_date, high_level, low_level, gender, secondhand, shipping_from, style])

csv_file.close()

Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success


KeyboardInterrupt: ignored

### Data cleaning

In [6]:
import pandas as pd

df = pd.read_excel('PrettyLittleThing_table.xls')
df.drop_duplicates(keep = 'first', inplace=True)
df['color'] = df['color'].str.replace('[','{').str.replace(']','}')

In [ ]:
GranCat = pd.read_csv("LowLevelCatagoryRegex.csv", header=None, skiprows=[0])

def process_cat(df):
    # get all the values in category regex
    subCats = GranCat.iloc[:, 1:]
    Cats = []
    for index, row in subCats.iterrows():
        for i in GranCat.columns[1:]:
            if pd.isnull(row[i]): continue
            else: Cats.append(row[i])
               
    # assign potential category for each clothing row based on title, else other
    clothesCat = []
    for index, row in df.iterrows():
        rowCat = float("NaN")
        for cat in Cats:
            if re.search(cat, row['display_name'].lower()):
                rowCat = cat
                print(rowCat)
                break
        if pd.isnull(rowCat):
            clothesCat.append('other')
        else:
            clothesCat.append(rowCat)

    return clothesCat

# set parent category to each product
def get_cat(x):
    if x == 'other': return x
    else: return [GranCat[0][i] for i in GranCat.index if x in GranCat.iloc[i].to_list()][0]

# get low_level column given a df
def get_lowlevel(df):
    df['low_level'] = process_cat(df)
    df['low_level'] = df['low_level'].apply(lambda x: get_cat(x))
    return df

df = get_lowlevel(df)

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
crop top
dresse?s?$
dresse?s?$
bikinis? ?-?bottoms?
crop top
sk[io]rts?
dresse?s?$
swimsuits?
shorts
bikinis? ?-?bottoms?
crop top
sk[io]rts?
sk[io]rts?
tops?
dresse?s?$
crop top
dresse?s?$
bodysuits?
sets?$
dresse?s?$
swimsuits?
jeans
shorts
dresse?s?$
hoodies?
jeans
jeans
sk[io]rts?
flares?
shorts
dresse?s?$
sets?$
fleece
shirts?
jumpsuits?
bra
[jl]eggings?
tops?
sweatshirts?
[jl]eggings?
crop top
pants?
bra
coats?
sweatshirts?
blazers?
jackets?
jumpsuits?
sets?$
sk[io]rts?
cami
pants?
dresse?s?$
dresse?s?
bodysuits?
dresse?s?$
sets?$
jeans
sk[io]rts?
dresse?s?$
dresse?s?$
cami
sets?$
bra
briefs?
shorts
bra
joggers?
shorts
crop top
bikinis? ?-?bottoms?
corsets?
bikinis? ?-?bottoms?
crop top
bra
bikinis? ?-?bottoms?
dresse?s?$
sk[io]rts?
bikinis? ?-?bottoms?
bikinis? ?-?bottoms?
crop top
sets?$
bikinis? ?-?bottoms?
shorts
bikinis? ?-?bottoms?
bikinis? ?-?bottoms?
bikinis? ?-?bottoms?
bikinis? ?-?top
pants?
d

In [ ]:
##### get mapped_id
GranRegs = pd.read_csv('MaterialProxy.csv', header=None, skiprows=[0])
GranRegs = GranRegs.drop(columns=[1])
GranRegs.columns = list(range(len(GranRegs.columns)))
#extract materials

#extract materials

def extractMaterials(df):
    fabric_text = df['materials']
    fabric_text = str(fabric_text).replace('solvent', '').replace('Grown', '').replace('grown', '').replace('organically', 'organic').replace('Organically', 'organic')
    splitted_fabric = fabric_text.replace("'", '').replace("[", '').replace("]", '').replace(",", '').replace('’', '').replace('.', '').replace('–', '').replace('-', '').split()
    pattern = "\d{1,3}[$%]"
    
    description_text = str(df['description']) + str(df['display_name']) + str(df['materials'])
    description_text = str(description_text).replace('solvent', '').replace('Grown', '').replace('grown', '').replace('organically', 'organic').replace('Organically', 'organic')
    splitted_description = description_text.replace("'", '').replace("[", '').replace("]", '').replace(",", '').replace('’', '').replace('.', '').replace('–', '').replace('-', '')
    pattern = "\d{1,3}[$%]"
    
    mat_list = []
    extra_materials = []
    
    
    ## getting the materials regex matching
    subRegs = GranRegs.iloc[:, 1:]
    Regs = []
    materialsReg = []
    for index, row in subRegs.iterrows():
        for i in range(1,len(GranRegs.columns)):
            if pd.isnull(row[i]): continue
            else: Regs.append(row[i])
    total_percentage = [] 
    is_no_percentage = True 
    for i in range(len(splitted_fabric)):
        if re.fullmatch(pattern,splitted_fabric[i]):
            total_percentage.append(float(splitted_fabric[i].replace("%", '')))
            if sum(total_percentage) <= 100:
            
                if splitted_fabric[i+1].lower() == 'organic':
                    mat_list.append(splitted_fabric[i] + ' ' + splitted_fabric[i+1].lower() + ' ' + splitted_fabric[i+2].lower())
                    is_no_percentage = False
                    
                elif splitted_fabric[i+1].lower() == 'recycled':
                    mat_list.append(splitted_fabric[i] + ' ' + splitted_fabric[i+1].lower() + ' ' + splitted_fabric[i+2].lower())
                    is_no_percentage = False
                
                elif splitted_fabric[i+1].lower() == 'tencel™':
                    mat_list.append(splitted_fabric[i] + ' ' + splitted_fabric[i+1].lower() + ' ' + splitted_fabric[i+2].lower())
                    is_no_percentage = False
                    
                else:
                    if splitted_fabric[i] + ' ' + splitted_fabric[i+1].lower() not in mat_list:
                        mat_list.append(splitted_fabric[i] + ' ' + splitted_fabric[i+1].lower())
                        is_no_percentage = False
                        
    if sum(total_percentage) < 100 or is_no_percentage == True:
        for reg in Regs:
            #print(splitted_description.lower())
            if re.search(reg, splitted_description.lower()):
                match = re.search(reg, splitted_description.lower())
                matched_material = splitted_description[match.start():match.end()].lower()
                
                if matched_material not in extra_materials:
                    extra_materials.append(matched_material)
    
    #print(extra_materials)
    if len(extra_materials)>1:
        if "cotton" in extra_materials and "organic cotton" in extra_materials:
            mat_list.append("100% organic cotton")
        if "leather" in extra_materials:
            mat_list.append("100% leather")
    elif len(extra_materials)!=0:
        mat_list.append("100% " + extra_materials[0])

            
    
    return mat_list


In [ ]:
df['materials'] = df.apply(lambda x: extractMaterials(x), axis=1)

In [7]:
df
df.to_csv('PrettyLittleThing_table.csv', encoding='utf-8-sig')